In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import glob
import re

from math import radians, sin, cos, acos, log, pi, tan, asin,sqrt
from decimal import Decimal
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON,STAMEN_TERRAIN
from ast import literal_eval
from scipy import stats
from math import radians, sin, cos, acos, log, pi, tan, asin,sqrt
from decimal import Decimal
from bokeh.embed import components
from bokeh.resources import CDN
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20b_20
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import gridplot

In [2]:
def merc(Coords):
    Coordinates = (Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * radians(lon)
    scale = x/lon
    y = 180.0/pi * log(tan(pi/4.0 + 
        lat * (pi/180.0)/2.0)) * scale
    return (x, y)

In [3]:
airbnb_cr = pd.read_csv('/home/novais/Downloads/airbnb_cr.csv')

In [4]:
listings = pd.read_csv('/home/novais/Downloads/listings.csv')

In [5]:
crimes = pd.read_csv('/home/novais/Downloads/crimes.csv')

In [6]:
crimes.head()

,Unnamed: 0,date,latitude,longitude,arrest,desc,lat,lon,location,loc,loc_id,month,day,year
0,0,2018-07-09,41.894328,-87.628143,1,BATTERY,41.89,-87.63,"(41.894327845999996, -87.62814321)","(41.89, -87.63)",0,7,9,2018
1,462,2018-07-09,41.906323,-87.767016,0,BATTERY,41.91,-87.77,"(41.906323375, -87.76701638799999)","(41.91, -87.77)",6280,7,9,2018
2,463,2018-07-09,41.762420,-87.683181,0,BATTERY,41.76,-87.68,"(41.762419957, -87.68318080799999)","(41.76, -87.68)",6039,7,9,2018
3,464,2018-07-09,41.794497,-87.647608,1,CRIMINAL DAMAGE,41.79,-87.65,"(41.794496999, -87.647607611)","(41.79, -87.65)",6105,7,9,2018
4,465,2018-07-09,41.903997,-87.643230,1,CRIMINAL DAMAGE,41.90,-87.64,"(41.903996883000005, -87.64323023799999)","(41.9, -87.64)",6274,7,9,2018


In [7]:
cr_count=crimes.groupby(['loc_id']).count()
cr_count=cr_count.desc.reset_index()
cr_count=cr_count.rename(index=str, columns={"desc": "crimes_count"})
cr_count.head()

,loc_id,crimes_count
0,0,3714
1,4728,2
2,4808,7
3,4884,1
4,4931,1


In [8]:
crimes2=crimes.merge(cr_count,on='loc_id')
crimes2=crimes2.groupby('loc_id').mean()
crimes2=crimes2.reset_index()
crimes2=crimes2[['lat','lon','loc_id','crimes_count']]
crimes2['loc']=list(zip(crimes2.lat,crimes2.lon))
crimes2.head()

,lat,lon,loc_id,crimes_count,loc
0,41.89,-87.63,0,3714.0,"(41.890000000001756, -87.63000000000348)"
1,41.65,-87.59,4728,2.0,"(41.65, -87.59)"
2,41.73,-87.72,4808,7.0,"(41.73, -87.72000000000001)"
3,41.68,-87.56,4884,1.0,"(41.68, -87.56)"
4,41.67,-87.57,4931,1.0,"(41.67, -87.57)"


In [9]:
crimes2['new_count'] = crimes2['crimes_count'].apply([lambda x: 5 if x<100 else(10 if x<500 else 20) ])

In [10]:
crimes2.head()

,lat,lon,loc_id,crimes_count,loc,new_count
0,41.89,-87.63,0,3714.0,"(41.890000000001756, -87.63000000000348)",20
1,41.65,-87.59,4728,2.0,"(41.65, -87.59)",5
2,41.73,-87.72,4808,7.0,"(41.73, -87.72000000000001)",5
3,41.68,-87.56,4884,1.0,"(41.68, -87.56)",5
4,41.67,-87.57,4931,1.0,"(41.67, -87.57)",5


In [11]:
crimes2.crimes_count.describe()

count     708.000000
mean      329.539548
std       393.078223
min         1.000000
25%        58.000000
50%       224.000000
75%       465.500000
max      4669.000000
Name: crimes_count, dtype: float64

In [12]:
# crimes2['location'] = crimes2['location'].apply(lambda x: literal_eval(x))
crimes2['coords_x'] = crimes2['loc'].apply(lambda x: merc(x)[0])
crimes2['coords_y'] = crimes2['loc'].apply(lambda x: merc(x)[1])

c1 = crimes2[crimes2['new_count']==5]
c2 = crimes2[crimes2['new_count']==10]
c3 = crimes2[crimes2['new_count']==20]

p1 = figure(title = "Crimes Count")
p1.xaxis.axis_label = 'Latitude'
p1.yaxis.axis_label = 'Longitude'

source1 = ColumnDataSource(data=dict(
    count=c1['crimes_count'],
    cx=c1['coords_x'],
    cy=c1['coords_y'],
    names = ['< 100']*len(c1),
    colors = ['blue']*len(c1),
    sizes = c1['new_count']))
source2 = ColumnDataSource(data=dict(
    count=c2['crimes_count'],
    cx=c2['coords_x'],
    cy=c2['coords_y'],
    names = ['[100,500]']*len(c2),
    colors = ['coral']*len(c2),
    sizes = c2['new_count']))
source3 = ColumnDataSource(data=dict(
    count=c3['crimes_count'],
    cx=c3['coords_x'],
    cy=c3['coords_y'],
    names = ['> 500']*len(c3),
    colors = ['maroon']*len(c3),
    sizes = c3['new_count']))


p1 = figure(x_axis_type="mercator", y_axis_type="mercator")
p1.add_tile(CARTODBPOSITRON)

p1.circle(x = 'cx',y = 'cy',source=source1,color='colors',legend='names',size='sizes',line_color="#000000",line_width=1)
p1.circle(x = 'cx',y = 'cy',source=source2,color='colors',legend='names',size='sizes',line_color="#000000",line_width=1)
p1.circle(x = 'cx',y = 'cy',source=source3,color='colors',legend='names',size='sizes',line_color="#000000",line_width=1)

p1.add_tools(HoverTool(
    tooltips=[
        ( 'count', '@count')
    ],
    
    formatters={
        'count' : 'numeral',   # use 'printf' formatter for 'adj close' field
                                  # use default 'numeral' formatter for other fields
    },

))

p1.legend.location = "top_right"
p1.legend.click_policy="hide"

p1_script, p1_div = components(p1)

In [13]:
output_notebook()


Loading BokehJS ...

In [15]:
show(p1)

In [ ]:
print(p1_script)

In [ ]:
print(p1_div)